# Introduction

Notebook para descrever o preprocessamento das imagens usadas pelas redes descritas no paper.

### Datasets utilizados

#### NUAA

O dataset NUAA foi um dos primeiros a ser publicamente disponivel para o problema de FAS, ele conta com por volta de 12000 imagens divididas em 16 identidades, seperadas em treino e teste. Por padrão há sobreposição de identidades entre o conjunto de treino e teste. Eventualmente nos nossos experimentos separamos as 16 identidades em 5+5+6 para treino, validação e teste, sendo estes conjuntos dijuntos.

O dataset conta apenas com ataques por foto. Tanto as capturas de ataque quanto as reais se dividem em até 3 sessões onde cada sessão é realizada num ambiente diferente, apesar disso nem todas as identidades tem imagens reais ou de atque em todas as sessões. O dataset em si não é bem padronizado em relação a quantas imagens por pessoa ou quantas imagens por sessão existem.

#### ReplayAttack

O replay attack é um dataset bastante citado na literatura, ele consiste de 1200 videos sendo 200 reais e 1000 ataques, divididos entre 50 identidades, separadas em treino, validação e teste, sem sobreposição. Cada video real tem por volta de 15s de duração enquanto os ataques tem quase 9,5s. Apos extraimos todos os frames dos videos temos aproximadamente 75000 iamgens reais e 235000 imagens de ataque. 

Os videos são gravados em dois cenarios: um controlado (fundo uniforme) e um descontrolado (fundo com quadro com reflexo). Os ataques são dados por displays de celular, displays de IPad, e fotos impressas. Todas as imagens de ataque foram capturadas com uma camera fotografica Cannon e todos os videos reais e de atque foram gravados com um MacBook Air.

#### MSU

Similarmente ao Replay Attack o MSU tambem é facilmente acessivel na internet, o dataset é coposto de 280 videos sendo 70 reais e 210 de ataque, com um total de 35 identidades divididas inicialmente em treino e teste. Os videos tem taxas de captura (FPS) e duração variados. Ao final do processo de extração de frames temos aproximadamente 19000 frames reais e 58000 frames de ataque.

As sessões de captura foram feitos em ambientes com fundos diferentes, os ataques foram realizados com dispositivos moveis, tables e impressões.

#### OULU

### Nenhum pre-processamento

Experimentos iniciais foram feitos entre os bancos de dados com e sem pre-processamento, o motivo dessa decisão se deu pelo fato de testar a importancia do background na identificação de um ataque, visto que este tem informações importantes como identificação de dedos ou bordas de dispositivos e papeis. Apesar de bons resultados sido obtidos, no fim optamos por fazer o pre-processamento.

### Motivações do pre-processamento

Apesar dos bons resultados obtidos com as imagens raw, identificamos que num cenario pratico ao trabalhar com diferentes resoluções de dispositivos de captura, com pessoas que podem estar a diferentes distancias da camera, a realização de um redimensionamento para o tamanho de entrada da rede poderia ser prejudicial. Isso ocorre pois a entrada da rede nos primeiros experimentos era 224x224, com isso distorções nas imagens eram geradas ao simplesmente fazer um redimensionamento. Essas distorções eram menos intensas em bancos cujo as imagens possuem baixa resolução (320x240 no replay attack). mas são altamente prejudiciais em imagens de resoluções maiores (1920x1080 no OULU). 

Alem disso outro problema é que algumas fotos foram tiradas em modo paisagem e outras em modo retrato, sendo assim, as distorções acabam por não seguir um padrão quando ocorrem. Por fim outra complicação é que em casos praticos a variação na distancia do usuario para a camera pode ocasionar em faces muito pequenas ou muito grandes, algo prejudicial para a adetecção.

Outra frente que cogitamos atacar em relação a pre-processamento foi a subamostragem dos bancos de imagens, enquanto quase todos os nossos bancos(exeto o NUAA) são compostos de video, a alta taxa de frames por segundo faz com que muitos frames sejam muito semelhantes uns aos outros. A fim de reduzir a carga computacional realizamos testes para subamostragem dos datasets

### Crop

A primeira e mais simples abordagem pensada para realização de pre-processamento de dados foi o recorte da reagião da face. As informações sobre as detecções foram dadas pelo detector da biblioteca Dlib compilada para C++, que identifica a "bounding box" da face. As regiões da face então eram cortadas e redimensionadas para o tamanho de entrada da rede (224x224).

Alem da realização do recorte da Face tambem foram realizados testes com a expansão da "bounding box" relativo a um fator. Essa abordagem nos permite incluir mais informações de fundo na imagem, assim auxiliando na identificação de bordas de dispositivos, ou na presença de dedos na imagem, duas caracteristicas comuns de ataques descuidados.

### Subsampling


Subamostragem no contexto do treinamento de redes neurais implica na partição de uma amostra maior em um conjunto menor, essa é uma tecnica frequentimente utilizada quando se há uma grande redundancia de informação, e seu principal proposito é a redução do custo computacional na etapa de treinamento de modelos. Como a maioria dos nossos datasets são dados primariamente na forma de video, o processo de extração de frames acaba por gerar uma grande quantidade de imagens com pouca ou quase nenhuma variação devida a alta taxa de FPS das gravações. Sendo assim em paralelo ao crop tambem pensamos numa tecnica de subamostragem onde não é preciso utilizar os bancos de dados por completo, mas mantendo o conteudo das imagens diferentes o suficiente para não perder variações significantes entre as imagens.

A abordagem mais comun a ser utilizada aqui seria simplesmente pegar um frame a cada X, esse metodod e pular frames é utilizado como uma medida rapida mas ele tem um problema. Por nossas observações pequenas variações podem ocorrer num intervalo de 2 a 4 frames, em seguida as amostras voltam a não ter variações e apenas adcionar redundancia para os dados. Uma abordagem simples de pular frames pode muito facilmente passar sobre essas rapidas variações e não incluir esses dados no treinamento.

Optamos então por realizar uma subamostragem mais refinada, utilizando um algoritimo de calculo de similaridade chamado de structural similarity index measure (SSIM). Calculando as similaridades das imagens podemos então pegar apenas as amostras mais diferentes, não só isso ainda é possivel manter o balanceamento entre classes e identidades dentro das classes. Para nossos experimentos vimos que subamostrando os datasets MSU, Replay Attack e OULU, para terem 6000 imagens de cada label foi satisfatorio. No fim cada dataset (exceto o NUAA que ja é relativamente pequeno) teve por volta de 12000 imagens de treino e a mesma quantidade para validação. Afim de manter os resultados comparaveis com a literatura o conjunto de teste foi mantido sem subamostragem.

### Alinhamento

Apesar de bons resultados mostrados com a subamostragem e recorte das faces percebemos que nosso modelo poderia ter um problema enfrentando cenarios reais. A abordagem de recorte e redimensionamento não produz resultados tão bons quando há uma grande variação nos tamanhos de face ou quando a face esta rotacionada, algo que é bem comum num cenario real devido a possibilidade de uma pessoa tirar uma foto a diferentes distanciamentos do dispositivo de captura e de varias maneiras diferentes. Para solucionar esse problema investigamos então possibilidades de novos metodos de pre-processamento, e optamos por realizar o alinhamento geometrico da face.

O detector da Dlib alem de fornecer informações sobre a "bounding box" da face tambem identifica a posição dos dois olhos, com essa informação podemos calcular a angulação de rotação da face, bem como a distancia entre os olhos. Esses dados podem ser utilizado spara calcular alem da rotação, a translação da face e a escala. Aplicando uma matriz de rotação sobre a imagem podemos obter então um alinhamento muito mais robusto.